In [1]:
import Waven.WaveletGenerator as wg
import Waven.Analysis_Utils as au
import Waven.LoadPinkNoise as lpn
import numpy as np
import gc
import os
import torch

In [2]:
# List of default parameters for the Gabor Library
gabor_param={
    "N_thetas":"8",
    "Sigmas": "[2, 3, 4, 5, 6, 8]",
    "Frequencies": "[0.015, 0.04, 0.07, 0.1]",
    "Phases": "[0, 90]",
    "NX": "135",
    "NY": "54",
    "Save Path":"/home/leonk/waven-working-/temp/gabors_library.npy"
}

# List of default parameters
param_defaults = {
    "Path Directory": "",
    "Dirs": "/home/leonk/waven-working-/temp/LE_ROS-2210_2025-11-28_scan9FXEU7TJ_sess9FXEU7TJ",
    "Experiment Info": ["0", "1", "2"],
    "Number of Planes": "1",
    "Block End": "0",
    "screen_x":"800",
    "screen_y":"600",
    "NX": "135",
    "NY": "54",
    "Resolution":"1.2",
    "Sigmas": "[2, 3, 4, 5, 6, 8]",
    "Frequencies": "[0.015, 0.04, 0.07, 0.1]",
    "Visual Coverage":"[-135, 45, 34, -34]",
    "Analysis Coverage": "[-135, 0, 34, -34]",
    "Number of Frames": "27300",
    "Number of Trials to Keep": "1",
    "Movie Path": "/home/leonk/waven-working-/temp/fullscreen_zebra.mp4",
    "Library Path": "/home/leonk/waven-working-/temp/gabors_library.npy",
    "Spks Path": "None"
}

In [3]:
# Non-GUI analysis workflow
import matplotlib.pyplot as plt
from pathlib import Path

def run_analysis(param_defaults, gabor_param):
    """
    Run analysis without GUI - suitable for Jupyter notebooks and headless environments
    """
    
    # Extract parameters
    path_directory = param_defaults["Path Directory"]
    dirs = [param_defaults["Dirs"]]
    exp_info = param_defaults["Experiment Info"]
    sigmas = np.array(eval(param_defaults["Sigmas"]))
    visual_coverage = eval(param_defaults["Visual Coverage"])
    analysis_coverage = eval(param_defaults["Analysis Coverage"])
    n_planes = int(param_defaults["Number of Planes"])
    block_end = int(param_defaults["Block End"])
    screen_x = int(param_defaults["screen_x"])
    screen_y = int(param_defaults["screen_y"])
    nx = int(param_defaults["NX"])
    ny = int(param_defaults["NY"])
    ns = len(sigmas)
    resolution = float(param_defaults["Resolution"])
    spks_path = param_defaults["Spks Path"]
    nb_frames = int(param_defaults["Number of Frames"])
    n_trial2keep = int(param_defaults["Number of Trials to Keep"])
    movpath = param_defaults["Movie Path"]
    lib_path = param_defaults["Library Path"]
    n_theta = int(gabor_param["N_thetas"])
    
    screen_ratio = abs(visual_coverage[0] - visual_coverage[1]) / nx
    xM, xm, yM, ym = analysis_coverage
    
    print(f"Visual coverage: {visual_coverage}, Sigmas: {sigmas}, NS: {ns}")
    print(f"Experiment info: {exp_info}")
    print(f"Directories: {dirs}")
    
    # Build paths
    pathdata = dirs[0] + '/' + exp_info[0] + '/' + exp_info[1] + '/' + str(exp_info[2])
    pathsuite2p = pathdata + '/suite2p'
    
    deg_per_pix = abs(xM - xm) / nx
    sigmas_deg = np.trunc(2 * deg_per_pix * sigmas * 100) / 100
    
    print(f"Data path: {pathdata}")
    print(f"Suite2p path: {pathsuite2p}")
    
    # Load spike data
    if spks_path == 'None':
        print('Aligning data...')
        spks, spks_n, neuron_pos = lpn.loadSPKMesoscope(exp_info, dirs, pathsuite2p, block_end, n_planes, nb_frames,
                                                        threshold=1.25, last=True, method='frame2ttl')
        neuron_pos = au.correctNeuronPos(neuron_pos, resolution)
        neuron_pos[:, 1] = abs(neuron_pos[:, 1] - np.max(neuron_pos[:, 1]))
    else:
        print(f'Loading spks file from {spks_path}')
        try:
            spks = np.load(spks_path)
            parent_dir = os.path.dirname(spks_path)
            neuron_pos = np.load(os.path.join(parent_dir, 'pos.npy'))
        except Exception as e:
            print(f"Error loading file: {e}")
            return None
    
    print(f"Spike data shape: {spks.shape}")
    print(f"Neuron positions shape: {neuron_pos.shape}")
    
    # Compute response reliability and skewness
    print("Computing response reliability...")
    respcorr = au.repetability_trial3(spks, neuron_pos, plotting=False)
    skewness = au.compute_skewness_neurons(spks, plotting=False)
    skewness = np.array(skewness)
    filter_mask = np.logical_and(respcorr >= 0.2, skewness <= 20)
    
    # Plot neuron positions
    fig1, ax1 = plt.subplots(figsize=(8, 8))
    ax1.scatter(neuron_pos[:, 0], neuron_pos[:, 1], c='k', alpha=0.5, s=30)
    ax1.set_xlabel('X position (um)')
    ax1.set_ylabel('Y position (um)')
    ax1.set_title("Neuron Positions")
    plt.tight_layout()
    plt.show()
    
    # Load wavelets
    parent_dir = os.path.dirname(movpath)
    print(f"Loading wavelets from {parent_dir}...")
    
    try:
        wavelets_downsampled = np.load(os.path.join(parent_dir, 'dwt_downsampled_videodata.npy'))
        w_r_downsampled = wavelets_downsampled[0]
        w_i_downsampled = wavelets_downsampled[1]
        w_c_downsampled = wavelets_downsampled[2]
        del wavelets_downsampled
        gc.collect()
        print("Loaded downsampled wavelets")
    except Exception as e:
        print(f"Downsampled wavelets not found: {e}")
        try:
            print("Attempting to load coarse wavelets...")
            w_r_downsampled, w_i_downsampled, w_c_downsampled = au.coarseWavelet(parent_dir, False, nx, ny, 27, 11,
                                                                                  n_theta, ns)
            print("Loaded coarse wavelets")
        except Exception as e:
            print(f"Error loading wavelets: {e}")
            print("Attempting full wavelet decomposition...")
            try:
                if (visual_coverage != analysis_coverage):
                    visual_coverage_arr = np.array(visual_coverage)
                    analysis_coverage_arr = np.array(analysis_coverage)
                    ratio_x = 1 - ((visual_coverage_arr[0] - visual_coverage_arr[1]) - (
                            analysis_coverage_arr[0] - analysis_coverage_arr[1])) / (
                                      visual_coverage_arr[0] - visual_coverage_arr[1])
                    ratio_y = 1 - ((visual_coverage_arr[2] - visual_coverage_arr[3]) - (
                            analysis_coverage_arr[2] - analysis_coverage_arr[3])) / (
                                      visual_coverage_arr[2] - visual_coverage_arr[3])
                else:
                    ratio_x = 1
                    ratio_y = 1
                
                wg.downsample_video_binary(movpath, visual_coverage, analysis_coverage, shape=(ny, nx), chunk_size=1000,
                                        ratios=(ratio_x, ratio_y))
                videodata = np.load(movpath[:-4] + '_downsampled.npy')
                wg.waveletDecomposition(videodata, 0, sigmas, parent_dir, library_path=lib_path)
                wg.waveletDecomposition(videodata, 1, sigmas, parent_dir, library_path=lib_path)
                
                w_r_downsampled, w_i_downsampled, w_c_downsampled = au.coarseWavelet(parent_dir, False, nx, ny, 27, 11,
                                                                                      n_theta, ns)
                print("Completed full wavelet decomposition")
            except Exception as e:
                print(f"Error in wavelet decomposition: {e}")
                return None
    
    # Compute receptive fields using Pearson correlation
    print("Computing receptive fields...")
    rfs_gabor = au.PearsonCorrelationPinkNoise(w_c_downsampled.reshape(18000, -1), 
                                                np.mean(spks[:, :18000], axis=0),
                                                neuron_pos, 27, 11, ns, analysis_coverage, 
                                                screen_ratio, sigmas_deg, plotting=True)
    
    # Plot retinotopy maps
    fig2, ax2 = plt.subplots(2, 2, figsize=(14, 12))
    maxes1 = rfs_gabor[2]
    plt.rcParams['axes.facecolor'] = 'none'
    
    m = ax2[0, 0].scatter(neuron_pos[:, 0], neuron_pos[:, 1], s=10, c=maxes1[0], cmap='jet', alpha=filter_mask)
    fig2.colorbar(m, ax=ax2[0, 0])
    ax2[0, 0].set_title('Azimuth Preference (deg)')
    ax2[0, 0].set_xlabel('X (um)')
    ax2[0, 0].set_ylabel('Y (um)')
    
    m = ax2[0, 1].scatter(neuron_pos[:, 0], neuron_pos[:, 1], s=10, c=maxes1[1], cmap='jet_r', alpha=filter_mask)
    fig2.colorbar(m, ax=ax2[0, 1])
    ax2[0, 1].set_title('Elevation Preference (deg)')
    ax2[0, 1].set_xlabel('X (um)')
    ax2[0, 1].set_ylabel('Y (um)')
    
    m = ax2[1, 0].scatter(neuron_pos[:, 0], neuron_pos[:, 1], s=10, c=maxes1[2], cmap='hsv', alpha=filter_mask)
    fig2.colorbar(m, ax=ax2[1, 0])
    ax2[1, 0].set_title('Orientation Preference (deg)')
    ax2[1, 0].set_xlabel('X (um)')
    ax2[1, 0].set_ylabel('Y (um)')
    
    m = ax2[1, 1].scatter(neuron_pos[:, 0], neuron_pos[:, 1], s=10, c=maxes1[3], cmap='coolwarm', alpha=filter_mask)
    fig2.colorbar(m, ax=ax2[1, 1])
    ax2[1, 1].set_title('Preferred Size (deg)')
    ax2[1, 1].set_xlabel('X (um)')
    ax2[1, 1].set_ylabel('Y (um)')
    
    plt.tight_layout()
    plt.show()
    
    # Save results
    print("Saving results...")
    np.save(os.path.join(parent_dir, 'correlation_matrix.npy'), rfs_gabor[0])
    np.save(os.path.join(parent_dir, 'maxes_indices.npy'), rfs_gabor[1])
    np.save(os.path.join(parent_dir, 'maxes_corrected.npy'), rfs_gabor[2])
    
    print("Analysis complete!")
    return {
        'spks': spks,
        'neuron_pos': neuron_pos,
        'rfs_gabor': rfs_gabor,
        'filter_mask': filter_mask,
        'respcorr': respcorr,
        'skewness': skewness
    }

# Run the analysis
print("Starting analysis pipeline...")
results = run_analysis(param_defaults, gabor_param)


Starting analysis pipeline...
Visual coverage: [-135, 45, 34, -34], Sigmas: [2 3 4 5 6 8], NS: 6
Experiment info: ['0', '1', '2']
Directories: ['/home/leonk/waven-working-/temp/LE_ROS-2210_2025-11-28_scan9FXEU7TJ_sess9FXEU7TJ']
Data path: /home/leonk/waven-working-/temp/LE_ROS-2210_2025-11-28_scan9FXEU7TJ_sess9FXEU7TJ/0/1/2
Suite2p path: /home/leonk/waven-working-/temp/LE_ROS-2210_2025-11-28_scan9FXEU7TJ_sess9FXEU7TJ/0/1/2/suite2p
Aligning data...
last session
single plane
single plane
shape spks :  (500, 27300)
neuron_pos spks :  (500, 2)
0/1/2/1_2_0_Timeline.mat
File could not be found! Be sure that cortex_lab_utils.expt_dirs() includes all valid directories.


UnboundLocalError: local variable 'filepath' referenced before assignment